In [1]:
# initialize sagemaker variables
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/bert-pytorch'

role = sagemaker.get_execution_role()

In [2]:
# download our trained model to the notebook runtime
!wget https://gradient-fire.s3.amazonaws.com/model.pth

--2020-02-28 15:26:32--  https://gradient-fire.s3.amazonaws.com/model.pth
Resolving gradient-fire.s3.amazonaws.com (gradient-fire.s3.amazonaws.com)... 52.216.204.227
Connecting to gradient-fire.s3.amazonaws.com (gradient-fire.s3.amazonaws.com)|52.216.204.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437983063 (418M) [application/x-www-form-urlencoded]
Saving to: ‘model.pth.1’

model.pth.1         100%[===================>] 417.69M  94.8MB/s    in 4.5s    

2020-02-28 15:26:36 (93.4 MB/s) - ‘model.pth.1’ saved [437983063/437983063]



In [23]:
# create a tar file from the model file
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('model.pth', recursive=True)

In [24]:
model_path = 'model.tar.gz'

In [25]:
# upload model artifacts to S3
model_artifact = sagemaker_session.upload_data(path=model_path, bucket=bucket, key_prefix=prefix)

In [32]:
model_artifact

In [33]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

# setup the RealTimePredictor object for serializing the inputs to tensors for pytorch
class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

# build the sagemaker model
model = PyTorchModel(model_data=model_artifact,
                     role = role,
                     framework_version='1.0.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)

In [34]:
# deploy the model as an endpoint
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------!

In [38]:
# make predictions
predictor.predict("Excellent movie i loved it")

b'0'